In [1]:
# -*- coding: utf-8 -*-
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("create df") \
    .getOrCreate()

# 第一组数据，包含年龄、体重、身高信息
body_info = [["Bom", 20, 97.6, 165],
             ["Alice", 23, 90.0, 160],
             ["Alice", 85, 99, 92],
             ["kuke", 33, 190.0, 170],
             ["kuke", 77, 82, 80],
             ["jike", 19, 120.0, 170],
             ["Joe", 24, 89.0, 162]]

body_df = spark.createDataFrame(body_info, ["name", "age", "weight", "height"])
body_df.show()

# 第二组数据，包含一些成绩信息，语数外
score_info = [["Bom", 88, 97, 90],
#              ["Alice", 85, 99, 92],
             ["kuke", 77, 82, 80],
#              ["jike", 65, 58, 30],
             ["Joe", 90, 100, 92]]
score_df = spark.createDataFrame(score_info, ["name", "Chinese", "Math", "English"])
score_df.show()

score_info = [["Bom"],
#              ["Alice", 85, 99, 92],
             ["kuke"],
              ["haha"],
              ["lis"],
#              ["jike", 65, 58, 30],
             ["Joe"]]
cc_df = spark.createDataFrame(score_info, ["name"])
score_df.show()

TypeError: field weight: Can not merge type <class 'pyspark.sql.types.DoubleType'> and <class 'pyspark.sql.types.LongType'>

In [10]:
# body_df.join(score_df, on="name", how="left").show()

cc_df.join(score_df.union(score_df), on="name", how="left").show()

+----+-------+----+-------+
|name|Chinese|Math|English|
+----+-------+----+-------+
| Bom|     88|  97|     90|
| Bom|     88|  97|     90|
| Joe|     90| 100|     92|
| Joe|     90| 100|     92|
|haha|   null|null|   null|
|kuke|     77|  82|     80|
|kuke|     77|  82|     80|
| lis|   null|null|   null|
+----+-------+----+-------+



In [24]:
# 第一组数据，包含年龄、体重、身高信息
body_info = [["Bom", 2, 55],
             ["Alice", 3, 33],
             ["kuke", 1, 22],
             ["jike", 1, 99],
             ["Joe", 4, 30]]

body_df = spark.createDataFrame(body_info, ["name", "age", "math" ])
body_df.show()

# # 第二组数据，包含一些成绩信息，语数外
# score_info = [["Bom", 88, 97, 90],
# #              ["Alice", 85, 99, 92],
#              ["kuke", 77, 82, 80],
# #              ["jike", 65, 58, 30],
#              ["Joe", 90, 100, 92]]
# score_df = spark.createDataFrame(score_info, ["name", "Chinese", "Math", "English"])
# score_df.show()

# score_info = [["Bom"],
# #              ["Alice", 85, 99, 92],
#              ["kuke"],
#               ["haha"],
#               ["lis"],
# #              ["jike", 65, 58, 30],
#              ["Joe"]]
# cc_df = spark.createDataFrame(score_info, ["name"])
# score_df.show()

from pyspark.sql.functions import col

def unpivot(df, keys):
    # 功能：数据宽变长
    # 参数说明 df:dataframe,  keys 待转换表中需要保留的主键key，以list[]类型传入
    # 转换是为了避免字段类不匹配，统一将数据转换为string类型，如果保证数据类型完全一致，可以省略该句
    df = df.select(*[col(_).astype("string") for _ in df.columns])
    cols = [_ for _ in df.columns if _ not in keys]
    stack_str = ','.join(map(lambda x: "'%s', `%s`" % (x, x), cols))
    # feature, value 转换后的列名，可自定义
    df = df.selectExpr(*keys, "stack(%s, %s) as (feature, value)" % (len(cols), stack_str))
    return df

cc = body_df.groupBy("name").pivot("age", [1,2,3,4,]).count()
cc.show()
unpivot(cc, ["name"]).show()

+-----+---+----+
| name|age|math|
+-----+---+----+
|  Bom|  2|  55|
|Alice|  3|  33|
| kuke|  1|  22|
| jike|  1|  99|
|  Joe|  4|  30|
+-----+---+----+

+-----+----+----+----+----+
| name|   1|   2|   3|   4|
+-----+----+----+----+----+
| jike|   1|null|null|null|
|  Bom|null|   1|null|null|
|  Joe|null|null|null|   1|
|Alice|null|null|   1|null|
| kuke|   1|null|null|null|
+-----+----+----+----+----+

+-----+-------+-----+
| name|feature|value|
+-----+-------+-----+
| jike|      1|    1|
| jike|      2| null|
| jike|      3| null|
| jike|      4| null|
|  Bom|      1| null|
|  Bom|      2|    1|
|  Bom|      3| null|
|  Bom|      4| null|
|  Joe|      1| null|
|  Joe|      2| null|
|  Joe|      3| null|
|  Joe|      4|    1|
|Alice|      1| null|
|Alice|      2| null|
|Alice|      3|    1|
|Alice|      4| null|
| kuke|      1|    1|
| kuke|      2| null|
| kuke|      3| null|
| kuke|      4| null|
+-----+-------+-----+

